In [ ]:
import requests
import zipfile
import os
from io import BytesIO
import pandas as pd
from tabulate import tabulate
import chardet

In [ ]:
uri = 'https://danepubliczne.imgw.pl/api/data/meteo/'
response = requests.get(uri)
if response.status_code == 200:
    print("Data fetched successfully.")
    print(f"Response body: {response.text}...")  # Print first 100 characters

In [ ]:
extract_dir = './data/raw'
years = range(2010, 2026)
samples = range(5, 1001, 5)

pro_data = './data/processed'

In [ ]:
def download_file(url, out_path):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()

    
        with open(out_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
            
        print(f"Plik pomyślnie pobrany i zapisany jako: {out_path}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"\n❌ Błąd podczas pobierania: {e}")
        return False
    except Exception as e:
        print(f"\n❌ Wystąpił nieoczekiwany błąd: {e}")
        return False
        
def handle_zip(zip_path, rm_zip=True):
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            extract_path = os.path.splitext(zip_path)[0]
            zip_ref.extractall(extract_path)
            print(f"Pliki zostały rozpakowane do: {extract_path}")
        if rm_zip:
            os.remove(zip_path)
            print(f"Plik ZIP został usunięty: {zip_path}")
        return True
    except zipfile.BadZipFile:
        print("❌ Błąd: Plik nie jest prawidłowym archiwum ZIP.")
        return False
    except Exception as e:
        print(f"\n❌ Wystąpił nieoczekiwany błąd podczas rozpakowywania: {e}")
        return False
        

In [ ]:
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir, exist_ok=True)

    for y in years:
        for s in samples:
            url = f"https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/terminowe/synop/{y}/{y}_{s}_s.zip"
            filename = url.split('/')[-1]
            out_path = os.path.join(extract_dir, filename)
            
            if download_file(url, out_path):
                handle_zip(out_path, rm_zip=True)
                print(f"Pobrano i rozpakowano dane dla roku {y} z {s} próbkami.")

In [ ]:
os.makedirs(extract_dir, exist_ok=True)

paths = []
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        if file.endswith('.csv'):
            full_path = os.path.join(root, file)
            paths.append(full_path)
            
print(f"\nZnaleziono {len(paths)} plików CSV.")

In [ ]:
with open(paths[0], 'rb') as f:
    raw_data = f.read(10000)
    
result = chardet.detect(raw_data)
detected_encoding = result['encoding']
print(f"Detected encoding: {detected_encoding}")

In [ ]:
column_names = [
    "Kod_stacji", "Nazwa_stacji", "Rok", "Miesiąc", "Dzień", "Godzina",
    "Wysokosc_chmur_kod", "Status_HPOD",
    "Wysokosc_podstawy_nizszej_m", "Status_HPON",
    "Wysokosc_podstawy_wyzszej_m", "Status_HPOW",
    "Wysokosc_podstawy_opis", "Pomiar_przyrzadem_1_P",
    "Pomiar_przyrzadem_2_P", "Widzialnosc_kod",
    "Status_WID", "Widzialnosc_operatora_m", "Status_WIDO",
    "Widzialnosc_automat_m", "Status_WIDA", "Zachmurzenie_ogolne_oktanty",
    "Status_NOG", "Kierunek_wiatru_stopnie", "Status_KRWR",
    "Predkosc_wiatru_ms", "Status_FWR", "Poryw_wiatru_ms",
    "Status_PORW", "Temperatura_powietrza_C", "Status_TEMP",
    "Temperatura_termometru_zwilzonego_C", "Status_TTZW",
    "Wskaznik_wentylacji", "Wskaznik_lodu",
    "Cisnienie_pary_wodnej_hPa", "Status_CPW",
    "Wilgotnosc_wzgledna_proc", "Status_WLGW",
    "Temperatura_punktu_rosy_C", "Status_TPTR",
    "Cisnienie_na_poziomie_stacji_hPa", "Status_PPPS",
    "Cisnienie_na_poziomie_morza_hPa", "Status_PPPM",
    "Charakterystyka_tendencji_kod", "Wartosc_tendencji", "Status_APP",
    "Opad_za_6_godzin_mm", "Status_WO6G",
    "Rodzaj_opadu_za_6_godzin_kod", "Status_ROPT",
    "Pogoda_biezaca_kod", "Pogoda_ubiegla_kod", "Zachmurzenie_niskie_oktanty",
    "Status_CLCM", "Chmury_CL_kod", "Status_CHCL", "Chmury_CL_tekstem",
    "Chmury_CM_kod", "Status_CHCM", "Chmury_CM_tekstem",
    "Chmury_CH_kod", "Status_CHCH", "Chmury_CH_tekstem",
    "Stan_gruntu_kod", "Status_SGRN",
    "Niedosyt_wilgotnosci_hPa", "Status_DEFI",
    "Uslonecznienie", "Status_USLN",
    "Wystapienie_rosy_01", "Status_ROSW",
    "Poryw_maksymalny_ms", "Status_PORK",
    "Godzina_porywu", "Minuta_porywu",
    "Temperatura_gruntu_minus5_C", "Status_TG05",
    "Temperatura_gruntu_minus10_C", "Status_TG10",
    "Temperatura_gruntu_minus20_C", "Status_TG20",
    "Temperatura_gruntu_minus50_C", "Status_TG50",
    "Temperatura_gruntu_minus100_C", "Status_TG100",
    "Temperatura_minimalna_12h_C", "Status_TMIN",
    "Temperatura_maksymalna_12h_C", "Status_TMAX",
    "Temperatura_minimalna_przy_gruncie_12h_C", "Status_TGMI",
    "Rownowaznik_wodny_sniegu_mmcm", "Status_RWSN",
    "Wysokosc_pokrywy_snieznej_cm", "Status_PKSN",
    "Wysokosc_swiezo_spadlego_sniegu_cm", "Status_HSS",
    "Wysokosc_sniegu_na_poletku_cm", "Status_GRSN",
    "Gatunek_sniegu_kod", "Uksztaltowanie_pokrywy_kod",
    "Wysokosc_probki_cm", "Status_HPRO",
    "Zapas_wody_w_sniegu_mm", "Status_CIPR"
]
dfs = []
for path in paths:
    try:
        df = pd.read_csv(path, encoding=detected_encoding, low_memory=False, names=column_names, header=None, sep=',')
        dfs.append(df)
        print(f"Pomyślnie wczytano plik: {path}")
    except Exception as e:
        print(f"❌ Błąd podczas wczytywania pliku {path}: {e}")

In [ ]:
text_cols = ['Nazwa_stacji', 'Wysokosc_podstawy_opis', 
                     'Chmury_CL_tekstem', 'Chmury_CM_tekstem', 'Chmury_CH_tekstem']

for df in dfs:
    for col in text_cols:
        if col in df.columns:
            bytes_data = df[col].astype(str).str.encode('iso-8859-1', errors='replace')
            df[col] = (
                bytes_data.str.decode('cp1250', errors='replace')
                .str.strip()
            )

In [ ]:
print(tabulate(dfs[0].head(), headers='keys', tablefmt='psql'))

In [ ]:
stacje = [tmp['Nazwa_stacji'].unique() for tmp in dfs]
stacje_flat = [item for sublist in stacje for item in sublist]
unique_stacje = set(stacje_flat)
print(f"\nŁączna liczba unikalnych stacji przed filtrowaniem: {len(unique_stacje)}")
stacje_sort = sorted(unique_stacje)
print("Lista unikalnych stacji (posortowana):")
for stacja in stacje_sort:
    print(stacja)


In [ ]:
dfs = [
    tmp[tmp['Nazwa_stacji'].str.contains('WROCŁAW', na=False)] for tmp in dfs
]

df = pd.concat(dfs, ignore_index=True)
df = df.dropna(subset=['Rok', 'Miesiąc', 'Dzień', 'Godzina'])


stacje = df['Nazwa_stacji'].unique()
print(f"Stacje: {stacje}")
print(f"\nŁączna liczba rekordów po filtrowaniu: {len(df)}")

In [ ]:
print("\nPodgląd danych:")
print(tabulate(df.describe(include='all'), headers='keys', tablefmt='psql'))
print(tabulate(df.head(), headers='keys', tablefmt='psql'))

In [ ]:
# Słownik do mapowania polskich nazw na angielskie, wymagane przez Pandas
column_map = {
    'Rok': 'year', 
    'Miesiąc': 'month', 
    'Dzień': 'day', 
    'Godzina': 'hour'
}

# 1. Tworzymy kopię DataFrame z tylko wymaganymi kolumnami
datetime_df = df[['Rok', 'Miesiąc', 'Dzień', 'Godzina']].copy()

# 2. Zmieniamy nazwy kolumn na nazwy oczekiwane przez Pandas
datetime_df.rename(columns=column_map, inplace=True)

# 3. Wykonujemy konwersję
daty = pd.to_datetime(datetime_df)

# Kontynuujemy Twój oryginalny kod
df['Data'] = daty
df.set_index('Data', inplace=True)
df.sort_index(inplace=True)

In [ ]:
os.makedirs(pro_data, exist_ok=True)
df.to_csv(os.path.join(pro_data, 'wroclaw_synop_data.csv'), index=True)